In [4]:

############################################################### 
# The Python code for the four examples with dependent sources are provided below.
############################################################### 
# Standard declarations
import  PySpice.Logging.Logging as Logging
logger = Logging.setup_logging()

from PySpice.Spice.Netlist import Circuit
from PySpice.Unit import *
############################################################### 

# Circuit definition of the object
circuit = Circuit('Circuit with CCVS')

# Circuit netlist
circuit.R(1, 1, 2,            2@u_Ω)
circuit.R(2, 2, circuit.gnd, 20@u_Ω)
circuit.R(3, 2, 3,            5@u_Ω)
circuit.R(4, 4, circuit.gnd, 10@u_Ω)
circuit.R(5, 4, 5,            2@u_Ω)
circuit.V(1, 1, circuit.gnd, 20@u_V)
circuit.V('test', 3, 4, 0@u_V)
circuit.H(1, 5, circuit.gnd, 'Vtest', 8)

# Simulation for DC point analysis
simulator = circuit.simulator(temperature=25, nominal_temperature=25)
analysis = simulator.operating_point()

#Showing end values of the simulation
for node in analysis.nodes.values(): 

    print('Node {}: {:4.1f} V'.format(str(node), float(node)))
    
for node in analysis.branches.values(): 

    print('Node {}: {:5.2f} A'.format(str(node), float(node))) ##########################################################


Node 5:  9.6 V
Node 4: 10.0 V
Node 3: 10.0 V
Node 2: 16.0 V
Node 1: 20.0 V
Node v1: -2.00 A
Node vtest:  1.20 A
Node h1:  0.20 A


In [ ]:
################################################################
# This example depicts how to simulate a regulated power supply circuit

################################################################
# STANDARD DECLARATIONS

import matplotlib.pyplot as plt
from matplotlib.widgets import Cursor
import numpy as np

import PySpice.Logging.Logging as Logging
logger = Logging.setup_logging()

from PySpice.Doc.ExampleTools import find_libraries
from PySpice.Probe.Plot import plot
from PySpice.Spice.Library import SpiceLibrary
from PySpice.Spice.Netlist import Circuit
from PySpice.Unit import *

#######################################################################
# CIRCUIT NETLIST AND SIMULATION

circuit = Circuit('Regulated Power Supply Circuit')
circuit.include("1N4002.lib")
circuit.include("1N751.lib")
from Transformer import Transformer

source = circuit.SinusoidalVoltageSource(1, 'pri', circuit.gnd, amplitude=339.41@u_V, frequency=50@u_Hz)

turn_ratio = 20
circuit.subcircuit(Transformer(turn_ratio))
circuit.X('transformer', 'Transformer', 'pri', circuit.gnd, 'sec_plus', 'sec_minus')

circuit.X('D1', '1N4002', 'sec_plus', 'smooth')
circuit.X('D2', '1N4002', circuit.gnd, 'sec_plus')
circuit.X('D3', '1N4002', circuit.gnd, 'sec_minus')
circuit.X('D4', '1N4002', 'sec_minus', 'smooth')

circuit.C('1', 'smooth', circuit.gnd, 1@u_mF)

circuit.R('Rs', 'smooth', 'output', 100@u_Ω)
circuit.X('Z', '1N751', circuit.gnd, 'output')
circuit.R('load', 'output', circuit.gnd, 1@u_kΩ)

simulator = circuit.simulator(temperature=25, nominal_temperature=25)
analysis = simulator.transient(step_time=1@u_us, end_time=60@u_ms)

#######################################################################
# PLOTTING COMMANDS

figure = plt.subplots(figsize=(11, 6))

axe = plt.subplot(211)
plt.title('Transformer Primary and Secondary')
plt.xlabel('Time [s]')
plt.ylabel('Voltage [V]')
plt.grid()
plot(analysis['pri'], axis=axe)
plot(analysis.sec_plus - analysis.sec_minus, axis=axe)
plt.legend(('primary', 'secondary'), loc=(.05,.1))
plt.ylim(float(-source.amplitude*1.1), float(source.amplitude*1.1))
axe.set_yticks([-340, 0, 340])

axe = plt.subplot(212)
plt.title('Regulated Power Supply Outputs')
plt.xlabel('Time [s]')
plt.ylabel('Voltage [V]')
plt.grid()
plot(analysis.sec_plus - analysis.sec_minus, axis=axe)
plot(analysis['smooth'], axis=axe)
plot(analysis['output'], axis=axe)
plt.legend(('secondary', 'filtered', 'output'), loc=(.05,.1))
plt.ylim(float(-source.amplitude*1.1/turn_ratio), float(source.amplitude*1.1/turn_ratio))
axe.set_yticks([-17, 0, 17])
cursor = Cursor(axe, useblit=True, color='red', linewidth=1)

plt.show()
#################################################################

In [ ]:
#######################################################################
# CIRCUIT NETLIST AND SIMULATION

circuit = Circuit('Full Wave Rectifier with Smoothing Capacitor')

circuit.model('1N4002', 'D', IS = 14.11E-11, N = 1.984, RS = 33.89E-3, IKF=94.81, XTITUN=3, KEG=1.110, CJO = 51.17E-12, M = .2762, VJ = .3905, FC = 0.5, ISR=100.0E-12, NR=2, BV=100.1, IBV=10, TT = 4.761E-6)

circuit.model('1N751', 'D', IS = 1.004E-15, N = 1, RS = .5875, IKF=0, XTITUN=3, EG=1.11, CJO = 160E-12, M = .5484, VJ = 0.75, FC = 0.5, ISR=1.8E-9, NR=2, BV=5.1, IBV=27.721E-3, NBV=1.1779)

source = circuit.SinusoidalVoltageSource(1, 'pri', circuit.gnd, amplitude=339.41@u_V, frequency=50@u_Hz)

turn_ratio = 20
circuit.subcircuit(Transformer(turn_ratio))
circuit.X('transformer', 'Transformer', 'pri', circuit.gnd, 'sec_plus', 'sec_minus')

circuit.D('D1', 'sec_plus', 'output',model='1N4002')
circuit.D('D2', circuit.gnd, 'sec_plus',model='1N4002')
circuit.D('D3', circuit.gnd, 'sec_minus',model='1N4002')
circuit.D('D4', 'sec_minus', 'output',model='1N4002')
circuit.R('load', 'output', circuit.gnd, 100@u_Ω)
circuit.C('1', 'output', circuit.gnd, 1@u_mF)

circuit.D('Z1', 'output', circuit.gnd, model='1N751')

simulator = circuit.simulator(temperature=25, nominal_temperature=25)
analysis = simulator.transient(step_time=1@u_us, end_time=80@u_ms)

In [ ]:
* -- spice --
* http://ltwiki.org/index.php?title=Standard.dio

*
* .model 1N4002 D (Is=14.11n N=1.984 Rs=33.89E-3 Ikf=94.81 Xti=3 Eg=1.110
*                   Cjo=51.17E-12 M=.2762 Vj=.3905 Fc=.5 Isr=100.0E-12 Nr=2
*                   Bv=100.1 Ibv=10 Tt=4.761E-6 Iave=1 Vpk=100 mfg=GI type=silicon)
*
*****************************************
*
* 1N4002

.SUBCKT 1N4002 1 2 

D1 1 2 1N4002
*
.MODEL 1N4002 D 
+ IS = 14.11E-11 
+ N = 1.984
+ RS = 33.89E-3
+ IKF=94.81
+ XTITUN=3
+ EG=1.110
+ CJO = 51.17E-12
+ M = .2762
+ VJ = .3905
+ FC = 0.5
+ ISR=100.0E-12
+ NR=2
+ BV=100.1
+ IBV=10
+ TT = 4.761E-6
.ENDS
*

In [ ]:
* -- spice --
* http://ltwiki.org/index.php?title=Standard.dio

*
*.model 1N751 D(Is=1.004f Rs=.5875 Ikf=0 N=1 Xti=3 Eg=1.11 Cjo=160p
*                M=.5484 Vj=.75 Fc=.5 Isr=1.8n Nr=2 Bv=5.1 Ibv=27.721m
*                Nbv=1.1779 Ibvl=1.1646m Nbvl=21.894 Tbv1=176.47u
*                Vpk=5.1 mfg=Motorola type=zener)
*
*****************************************
*
* 1N751

.SUBCKT 1N751 1 2 

D1 1 2 1N751
*
.MODEL 1N751 D 
+ IS = 1.004E-15
+ N = 1
+ RS = .5875 
+ IKF=0
+ XTITUN=3
+ EG=1.11
+ CJO = 160E-12
+ M = .5484 
+ VJ = 0.75
+ FC = 0.5
+ ISR=1.8E-9
+ NR=2
+ BV=5.1
+ IBV=27.721E-3
+ NBV=1.1779
.ENDS
*